In [72]:
! pip install plotly --quiet
! pip install pyarrow --quiet
! pip install statsmodels --quiet

import duckdb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller

In [73]:
con_gdelt = duckdb.connect('eda-ddb/eda-gdelt.ddb')
print(con_gdelt.execute("SELECT * FROM gdelt LIMIT 1").fetch_df().columns)
con_gdelt.close()

con_yfinance = duckdb.connect('eda-ddb/eda-yfinance.ddb')
print(con_yfinance.execute("SELECT * FROM yfinance LIMIT 1").fetch_df().columns)
con_yfinance.close()

Index(['GlobalEventID', 'Day', 'MonthYear', 'Year', 'FractionDate',
       'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode',
       'Actor2Code', 'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode',
       'IsRootEvent', 'EventCode', 'EventBaseCode', 'EventRootCode',
       'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources',
       'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_CountryCode',
       'Actor2Geo_Type', 'Actor2Geo_CountryCode', 'ActionGeo_Type',
       'ActionGeo_CountryCode', 'SOURCEURL'],
      dtype='object')
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Sector',
       'Stock', 'Date'],
      dtype='object')


---

# Question 4
What is the average delay in an event happening and the consequences/effect on the stock market (if any)?

### Granger Causality Test

H0: Time series x does not Granger-cause time series y $\\$
HA: Time series x Granger-causes time series y
- It uses to determine whether ot not one time series is useful for forcating another
- It requires the tested variable sequence to be a stationary (one whose properties do not depend on the time at which the series is observed)

### Augmented Dickey-Fuller Test
H0: The time series is non-stationary $\\$
HA: The time series is stationary
- It uses to see if a time series is stationary

---

# 2019 - 2024

## Monthly

### Load Data from GDELT

In [74]:
# Connect to DuckDB (create a single connection)
con_gdelt = duckdb.connect('eda-ddb/eda-gdelt.ddb')

# Load and aggregate GDELT data by month for 2019
gdelt_monthly_df_2019 = con_gdelt.execute("""
    SELECT 
        STRFTIME(STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d'), '%Y-%m') AS Month,
        COUNT(GlobalEventID) AS EventCount,
        AVG(AvgTone) AS AvgTone
    FROM gdelt
    WHERE STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d') BETWEEN '2019-01-01' AND '2019-12-31'
    GROUP BY Month
""").fetch_df()

# Load and aggregate GDELT data by month for 2020
gdelt_monthly_df_2020 = con_gdelt.execute("""
    SELECT 
        STRFTIME(STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d'), '%Y-%m') AS Month,
        COUNT(GlobalEventID) AS EventCount,
        AVG(AvgTone) AS AvgTone
    FROM gdelt
    WHERE STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d') BETWEEN '2020-01-01' AND '2020-12-31'
    GROUP BY Month
""").fetch_df()

# Load and aggregate GDELT data by month for 2021
gdelt_monthly_df_2021 = con_gdelt.execute("""
    SELECT 
        STRFTIME(STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d'), '%Y-%m') AS Month,
        COUNT(GlobalEventID) AS EventCount,
        AVG(AvgTone) AS AvgTone
    FROM gdelt
    WHERE STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d') BETWEEN '2021-01-01' AND '2021-12-31'
    GROUP BY Month
""").fetch_df()

# Load and aggregate GDELT data by month for 2022
gdelt_monthly_df_2022 = con_gdelt.execute("""
    SELECT 
        STRFTIME(STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d'), '%Y-%m') AS Month,
        COUNT(GlobalEventID) AS EventCount,
        AVG(AvgTone) AS AvgTone
    FROM gdelt
    WHERE STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d') BETWEEN '2022-01-01' AND '2022-12-31'
    GROUP BY Month
""").fetch_df()

# Load and aggregate GDELT data by month for 2023
gdelt_monthly_df_2023 = con_gdelt.execute("""
    SELECT 
        STRFTIME(STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d'), '%Y-%m') AS Month,
        COUNT(GlobalEventID) AS EventCount,
        AVG(AvgTone) AS AvgTone
    FROM gdelt
    WHERE STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d') BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY Month
""").fetch_df()

# Load and aggregate GDELT data by month for 2024
gdelt_monthly_df_2024 = con_gdelt.execute("""
    SELECT 
        STRFTIME(STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d'), '%Y-%m') AS Month,
        COUNT(GlobalEventID) AS EventCount,
        AVG(AvgTone) AS AvgTone
    FROM gdelt
    WHERE STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d') BETWEEN '2024-01-01' AND '2024-12-31'
    GROUP BY Month
""").fetch_df()

# Close the GDELT connection
con_gdelt.close()

### Load Data from YFinance

In [75]:
# Connect to Yahoo Finance database
con_yfinance = duckdb.connect('eda-ddb/eda-yfinance.ddb')

# Load and aggregate Yahoo Finance data by month for 2019
yfinance_monthly_df_2019 = con_yfinance.execute("""
    SELECT 
        STRFTIME(Date, '%Y-%m') AS Month,
        Stock,
        Sector,
        AVG(Close) AS AvgClose
    FROM yfinance 
    WHERE Date BETWEEN '2019-01-01' AND '2019-12-31'
    GROUP BY Month, Stock, Sector
""").fetch_df()

# Load and aggregate Yahoo Finance data by month for 2020
yfinance_monthly_df_2020 = con_yfinance.execute("""
    SELECT 
        STRFTIME(Date, '%Y-%m') AS Month,
        Stock,
        Sector,
        AVG(Close) AS AvgClose
    FROM yfinance 
    WHERE Date BETWEEN '2020-01-01' AND '2020-12-31'
    GROUP BY Month, Stock, Sector
""").fetch_df()

# Load and aggregate Yahoo Finance data by month for 2021
yfinance_monthly_df_2021 = con_yfinance.execute("""
    SELECT 
        STRFTIME(Date, '%Y-%m') AS Month,
        Stock,
        Sector,
        AVG(Close) AS AvgClose
    FROM yfinance 
    WHERE Date BETWEEN '2021-01-01' AND '2021-12-31'
    GROUP BY Month, Stock, Sector
""").fetch_df()

# Load and aggregate Yahoo Finance data by month for 2022
yfinance_monthly_df_2022 = con_yfinance.execute("""
    SELECT 
        STRFTIME(Date, '%Y-%m') AS Month,
        Stock,
        Sector,
        AVG(Close) AS AvgClose
    FROM yfinance 
    WHERE Date BETWEEN '2022-01-01' AND '2022-12-31'
    GROUP BY Month, Stock, Sector
""").fetch_df()

# Load and aggregate Yahoo Finance data by month for 2023
yfinance_monthly_df_2023 = con_yfinance.execute("""
    SELECT 
        STRFTIME(Date, '%Y-%m') AS Month,
        Stock,
        Sector,
        AVG(Close) AS AvgClose
    FROM yfinance 
    WHERE Date BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY Month, Stock, Sector
""").fetch_df()

# Load and aggregate Yahoo Finance data by month for 2024
yfinance_monthly_df_2024 = con_yfinance.execute("""
    SELECT 
        STRFTIME(Date, '%Y-%m') AS Month,
        Stock,
        Sector,
        AVG(Close) AS AvgClose
    FROM yfinance 
    WHERE Date BETWEEN '2024-01-01' AND '2024-12-31'
    GROUP BY Month, Stock, Sector
""").fetch_df()

# Close the Yahoo Finance connection
con_yfinance.close()

In [76]:
gdelt_monthly_df_2024

,Month,EventCount,AvgTone
0,2024-08,3630664,-2.202983
1,2024-04,3902125,-2.109203
2,2024-02,3858148,-2.011047
3,2024-01,3825620,-2.171162
4,2024-06,3597486,-1.863340
5,2024-09,3646906,-1.979349
6,2024-03,3983869,-1.901982
7,2024-07,3642451,-1.851922
8,2024-10,1875034,-2.010806
9,2024-05,3975562,-1.993838


In [77]:
yfinance_monthly_df_2024

,Month,Stock,Sector,AvgClose
0,2024-10,ADM,agriculture,58.256001
1,2024-08,CTVA,agriculture,53.467727
2,2024-09,CTVA,agriculture,56.774500
3,2024-06,DE,agriculture,373.718948
4,2024-03,BP,energy,37.091500
...,...,...,...,...
245,2024-06,T,telecom,18.203684
246,2024-09,TMUS,telecom,200.570000
247,2024-05,VZ,telecom,39.809092
248,2024-07,VZ,telecom,40.823182


### Merge

In [78]:
merged_monthly_df_2019 = pd.merge(gdelt_monthly_df_2019, yfinance_monthly_df_2019, on='Month', how='inner')
merged_monthly_df_2019['Month'] = pd.to_datetime(merged_monthly_df_2019['Month'])

merged_monthly_df_2020 = pd.merge(gdelt_monthly_df_2020, yfinance_monthly_df_2020, on='Month', how='inner')
merged_monthly_df_2020['Month'] = pd.to_datetime(merged_monthly_df_2020['Month'])

merged_monthly_df_2021 = pd.merge(gdelt_monthly_df_2021, yfinance_monthly_df_2021, on='Month', how='inner')
merged_monthly_df_2021['Month'] = pd.to_datetime(merged_monthly_df_2021['Month'])

merged_monthly_df_2022 = pd.merge(gdelt_monthly_df_2022, yfinance_monthly_df_2022, on='Month', how='inner')
merged_monthly_df_2022['Month'] = pd.to_datetime(merged_monthly_df_2022['Month'])

merged_monthly_df_2023 = pd.merge(gdelt_monthly_df_2023, yfinance_monthly_df_2023, on='Month', how='inner')
merged_monthly_df_2023['Month'] = pd.to_datetime(merged_monthly_df_2023['Month'])

merged_monthly_df_2024 = pd.merge(gdelt_monthly_df_2024, yfinance_monthly_df_2024, on='Month', how='inner')
merged_monthly_df_2024['Month'] = pd.to_datetime(merged_monthly_df_2024['Month'])

# Merge all the monthly dataframes
merged_monthly_df_all = pd.concat([
    merged_monthly_df_2019,
    merged_monthly_df_2020,
    merged_monthly_df_2021,
    merged_monthly_df_2022,
    merged_monthly_df_2023,
    merged_monthly_df_2024
])

In [79]:
# Group by sector then find the average close price for each month
sector_avg_close_df_2019 = merged_monthly_df_2019.groupby(['Month', 'Sector'], as_index=False).agg(
    {'AvgClose': 'mean', 'EventCount': 'sum'}
)
sector_avg_close_df_2020 = merged_monthly_df_2020.groupby(['Month', 'Sector'], as_index=False).agg(
    {'AvgClose': 'mean', 'EventCount': 'sum'}
)
sector_avg_close_df_2021 = merged_monthly_df_2021.groupby(['Month', 'Sector'], as_index=False).agg(
    {'AvgClose': 'mean', 'EventCount': 'sum'}
)
sector_avg_close_df_2022 = merged_monthly_df_2022.groupby(['Month', 'Sector'], as_index=False).agg(
    {'AvgClose': 'mean', 'EventCount': 'sum'}
)
sector_avg_close_df_2023 = merged_monthly_df_2023.groupby(['Month', 'Sector'], as_index=False).agg(
    {'AvgClose': 'mean', 'EventCount': 'sum'}
)
sector_avg_close_df_2024 = merged_monthly_df_2024.groupby(['Month', 'Sector'], as_index=False).agg(
    {'AvgClose': 'mean', 'EventCount': 'sum'}
)

# Merge all sector_avg_close_df dataframes
sector_avg_close_df_all = pd.concat([
    sector_avg_close_df_2019, 
    sector_avg_close_df_2020, 
    sector_avg_close_df_2021, 
    sector_avg_close_df_2022, 
    sector_avg_close_df_2023, 
    sector_avg_close_df_2024
])

sector_avg_close_df_all

,Month,Sector,AvgClose,EventCount
0,2019-01-01,agriculture,100.262619,9487062
1,2019-01-01,energy,72.409762,18974124
2,2019-01-01,finance,145.137937,14230593
3,2019-01-01,food,83.045079,14230593
4,2019-01-01,health,84.916431,14230593
...,...,...,...,...
75,2024-10-01,food,137.096333,5625102
76,2024-10-01,health,128.227333,5625102
77,2024-10-01,materials,52.503333,5625102
78,2024-10-01,tech,257.278001,5625102


### Time-Series Plot of Events and Stock Prices Monthly

In [98]:
# Create subplots with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Define unique sectors for color separation
sectors = sector_avg_close_df_all['Sector'].unique()

# Create a color palette to assign different colors to each sector
colors = {
    sector: color
    for sector, color in zip(sectors, px.colors.qualitative.Plotly)
}

# Add traces for each sector for AvgClose (primary y-axis)
for sector in sectors:
    sector_data = sector_avg_close_df_all[sector_avg_close_df_all['Sector'] == sector]
    # Plot AvgClose for the sector
    fig.add_trace(
        go.Scatter(
            x=sector_data['Month'], 
            y=sector_data['AvgClose'], 
            mode='lines+markers',
            name=f'{sector} AvgClose',
            line=dict(color=colors[sector]),
        ),
        secondary_y=False  # Primary y-axis
    )

    # Plot EventCount for the sector (secondary y-axis)
    fig.add_trace(
        go.Scatter(
            x=sector_data['Month'], 
            y=sector_data['EventCount'], 
            mode='lines+markers',
            name=f'{sector} EventCount',
            line=dict(dash='dot', color=colors[sector]),  # Dashed line for EventCount
        ),
        secondary_y=True  # Secondary y-axis
    )

# Update y-axis titles
fig.update_yaxes(title_text="AvgClose (Stock Price)", secondary_y=False)
fig.update_yaxes(title_text="EventCount", secondary_y=True)

# Update x-axis title and overall title
fig.update_layout(
    autosize=False,
    width=1500,
    height=800,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    title_text="Average Close Price and Event Count by Sector",
    xaxis_title="Year",
    legend_title="Sector",
)

# Show the plot
fig.show()

### Testing the Hypothesis

#### Augmented Dickey-Fuller Test

In [81]:
adfuller(merged_monthly_df_all['AvgClose'], autolag='AIC')

(-5.569523556108347,
 1.4783247557703534e-06,
 20,
 1723,
 {'1%': -3.434150968697906,
  '5%': -2.8632189151648864,
  '10%': -2.567663807484617},
 20523.35778708723)

In [82]:
adfuller(merged_monthly_df_all['EventCount'], autolag='AIC')

(-2.9827343538690543,
 0.03653855725861969,
 25,
 1718,
 {'1%': -3.4341620474831775,
  '5%': -2.863223805622509,
  '10%': -2.5676664115487458},
 45234.40223809878)

#### Granger Causality Test

In [83]:
data_lag = merged_monthly_df_all[['EventCount', 'AvgClose']].dropna()
granger_test_result = grangercausalitytests(data_lag, maxlag=12, verbose=True)

p_values = {lag: test[0]['ssr_ftest'][1] for lag, test in granger_test_result.items()}
p_values_df = pd.DataFrame(p_values.items(), columns=['Lag', 'p-value'])
p_values_df
# top_5_results = p_values_df.sort_values(by='p-value')
# top_5_results


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.4112  , p=0.5214  , df_denom=1740, df_num=1
ssr based chi2 test:   chi2=0.4119  , p=0.5210  , df=1
likelihood ratio test: chi2=0.4119  , p=0.5210  , df=1
parameter F test:         F=0.4112  , p=0.5214  , df_denom=1740, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.1371  , p=0.3210  , df_denom=1737, df_num=2
ssr based chi2 test:   chi2=2.2807  , p=0.3197  , df=2
likelihood ratio test: chi2=2.2792  , p=0.3199  , df=2
parameter F test:         F=1.1371  , p=0.3210  , df_denom=1737, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.2528  , p=0.2891  , df_denom=1734, df_num=3
ssr based chi2 test:   chi2=3.7737  , p=0.2870  , df=3
likelihood ratio test: chi2=3.7696  , p=0.2874  , df=3
parameter F test:         F=1.2528  , p=0.2891  , df_denom=1734, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.9952  , p=0.

C:\Users\lklin\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning:

verbose is deprecated since functions should not print results



,Lag,p-value
0,1,0.521442
1,2,0.320997
2,3,0.289098
3,4,0.408916
4,5,0.389084
5,6,0.341532
6,7,0.405621
7,8,0.469842
8,9,0.429110
9,10,0.373536


## Daily

### Load Data from GDELT

In [84]:
# Connect to DuckDB (create a single connection)
con_gdelt = duckdb.connect('eda-ddb/eda-gdelt.ddb')

# Load and aggregate GDELT data by day for 2019
gdelt_daily_df_2019 = con_gdelt.execute("""
    SELECT 
        STRFTIME(STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d'), '%Y-%m-%d') AS Day,
        COUNT(GlobalEventID) AS EventCount,
        AVG(AvgTone) AS AvgTone
    FROM gdelt
    WHERE STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d') BETWEEN '2019-01-01' AND '2019-12-31'
    GROUP BY Day
""").fetch_df()

# Load and aggregate GDELT data by day for 2020
gdelt_daily_df_2020 = con_gdelt.execute("""
    SELECT 
        STRFTIME(STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d'), '%Y-%m-%d') AS Day,
        COUNT(GlobalEventID) AS EventCount,
        AVG(AvgTone) AS AvgTone
    FROM gdelt
    WHERE STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d') BETWEEN '2020-01-01' AND '2020-12-31'
    GROUP BY Day
""").fetch_df()

# Load and aggregate GDELT data by day for 2021
gdelt_daily_df_2021 = con_gdelt.execute("""
    SELECT 
        STRFTIME(STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d'), '%Y-%m-%d') AS Day,
        COUNT(GlobalEventID) AS EventCount,
        AVG(AvgTone) AS AvgTone
    FROM gdelt
    WHERE STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d') BETWEEN '2021-01-01' AND '2021-12-31'
    GROUP BY Day
""").fetch_df()

# Load and aggregate GDELT data by day for 2022
gdelt_daily_df_2022 = con_gdelt.execute("""
    SELECT 
        STRFTIME(STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d'), '%Y-%m-%d') AS Day,
        COUNT(GlobalEventID) AS EventCount,
        AVG(AvgTone) AS AvgTone
    FROM gdelt
    WHERE STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d') BETWEEN '2022-01-01' AND '2022-12-31'
    GROUP BY Day
""").fetch_df()

# Load and aggregate GDELT data by day for 2023
gdelt_daily_df_2023 = con_gdelt.execute("""
    SELECT 
        STRFTIME(STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d'), '%Y-%m-%d') AS Day,
        COUNT(GlobalEventID) AS EventCount,
        AVG(AvgTone) AS AvgTone
    FROM gdelt
    WHERE STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d') BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY Day
""").fetch_df()

# Load and aggregate GDELT data by day for 2024
gdelt_daily_df_2024 = con_gdelt.execute("""
    SELECT 
        STRFTIME(STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d'), '%Y-%m-%d') AS Day,
        COUNT(GlobalEventID) AS EventCount,
        AVG(AvgTone) AS AvgTone
    FROM gdelt
    WHERE STRPTIME(CAST(Day AS VARCHAR), '%Y%m%d') BETWEEN '2024-01-01' AND '2024-12-31'
    GROUP BY Day
""").fetch_df()

# Close the GDELT connection
con_gdelt.close()

### Load Data from YFinance

In [85]:
# Connect to Yahoo Finance database
con_yfinance = duckdb.connect('eda-ddb/eda-yfinance.ddb')

# Load and aggregate Yahoo Finance data by day for 2019
yfinance_daily_df_2019 = con_yfinance.execute("""
    SELECT 
        STRFTIME(Date, '%Y-%m-%d') AS Day,
        Stock,
        Sector,
        AVG(Close) AS AvgClose
    FROM yfinance 
    WHERE Date BETWEEN '2019-01-01' AND '2019-12-31'
    GROUP BY Day, Stock, Sector
""").fetch_df()

# Load and aggregate Yahoo Finance data by day for 2020
yfinance_daily_df_2020 = con_yfinance.execute("""
    SELECT 
        STRFTIME(Date, '%Y-%m-%d') AS Day,
        Stock,
        Sector,
        AVG(Close) AS AvgClose
    FROM yfinance 
    WHERE Date BETWEEN '2020-01-01' AND '2020-12-31'
    GROUP BY Day, Stock, Sector
""").fetch_df()

# Load and aggregate Yahoo Finance data by day for 2021
yfinance_daily_df_2021 = con_yfinance.execute("""
    SELECT 
        STRFTIME(Date, '%Y-%m-%d') AS Day,
        Stock,
        Sector,
        AVG(Close) AS AvgClose
    FROM yfinance 
    WHERE Date BETWEEN '2021-01-01' AND '2021-12-31'
    GROUP BY Day, Stock, Sector
""").fetch_df()

# Load and aggregate Yahoo Finance data by day for 2022
yfinance_daily_df_2022 = con_yfinance.execute("""
    SELECT 
        STRFTIME(Date, '%Y-%m-%d') AS Day,
        Stock,
        Sector,
        AVG(Close) AS AvgClose
    FROM yfinance 
    WHERE Date BETWEEN '2022-01-01' AND '2022-12-31'
    GROUP BY Day, Stock, Sector
""").fetch_df()

# Load and aggregate Yahoo Finance data by day for 2023
yfinance_daily_df_2023 = con_yfinance.execute("""
    SELECT 
        STRFTIME(Date, '%Y-%m-%d') AS Day,
        Stock,
        Sector,
        AVG(Close) AS AvgClose
    FROM yfinance 
    WHERE Date BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY Day, Stock, Sector
""").fetch_df()

# Load and aggregate Yahoo Finance data by day for 2024
yfinance_daily_df_2024 = con_yfinance.execute("""
    SELECT 
        STRFTIME(Date, '%Y-%m-%d') AS Day,
        Stock,
        Sector,
        AVG(Close) AS AvgClose
    FROM yfinance 
    WHERE Date BETWEEN '2024-01-01' AND '2024-12-31'
    GROUP BY Day, Stock, Sector
""").fetch_df()

# Close the Yahoo Finance connection
con_yfinance.close()

### Merge

In [86]:
merged_daily_df_2019 = pd.merge(gdelt_daily_df_2019, yfinance_daily_df_2019, on='Day', how='inner')
merged_daily_df_2019['Day'] = pd.to_datetime(merged_daily_df_2019['Day'])

merged_daily_df_2020 = pd.merge(gdelt_daily_df_2020, yfinance_daily_df_2020, on='Day', how='inner')
merged_daily_df_2020['Day'] = pd.to_datetime(merged_daily_df_2020['Day'])

merged_daily_df_2021 = pd.merge(gdelt_daily_df_2021, yfinance_daily_df_2021, on='Day', how='inner')
merged_daily_df_2021['Day'] = pd.to_datetime(merged_daily_df_2021['Day'])

merged_daily_df_2022 = pd.merge(gdelt_daily_df_2022, yfinance_daily_df_2022, on='Day', how='inner')
merged_daily_df_2022['Day'] = pd.to_datetime(merged_daily_df_2022['Day'])

merged_daily_df_2023 = pd.merge(gdelt_daily_df_2023, yfinance_daily_df_2023, on='Day', how='inner')
merged_daily_df_2023['Day'] = pd.to_datetime(merged_daily_df_2023['Day'])

merged_daily_df_2024 = pd.merge(gdelt_daily_df_2024, yfinance_daily_df_2024, on='Day', how='inner')
merged_daily_df_2024['Day'] = pd.to_datetime(merged_daily_df_2024['Day'])

# Merge all the daily dataframes
merged_daily_df_all = pd.concat([
    merged_daily_df_2019,
    merged_daily_df_2020,
    merged_daily_df_2021,
    merged_daily_df_2022,
    merged_daily_df_2023,
    merged_daily_df_2024
])

In [87]:
# Group by sector then find the average close price for each day
sector_avg_close_daily_df_2019 = merged_daily_df_2019.groupby(['Day', 'Sector'], as_index=False).agg(
    {'AvgClose': 'mean', 'EventCount': 'sum'}
)
sector_avg_close_daily_df_2020 = merged_daily_df_2020.groupby(['Day', 'Sector'], as_index=False).agg(
    {'AvgClose': 'mean', 'EventCount': 'sum'}
)
sector_avg_close_daily_df_2021 = merged_daily_df_2021.groupby(['Day', 'Sector'], as_index=False).agg(
    {'AvgClose': 'mean', 'EventCount': 'sum'}
)
sector_avg_close_daily_df_2022 = merged_daily_df_2022.groupby(['Day', 'Sector'], as_index=False).agg(
    {'AvgClose': 'mean', 'EventCount': 'sum'}
)
sector_avg_close_daily_df_2023 = merged_daily_df_2023.groupby(['Day', 'Sector'], as_index=False).agg(
    {'AvgClose': 'mean', 'EventCount': 'sum'}
)
sector_avg_close_daily_df_2024 = merged_daily_df_2024.groupby(['Day', 'Sector'], as_index=False).agg(
    {'AvgClose': 'mean', 'EventCount': 'sum'}
)

# Merge all sector_avg_close_daily_df dataframes
sector_avg_close_daily_df_all = pd.concat([
    sector_avg_close_daily_df_2019, 
    sector_avg_close_daily_df_2020, 
    sector_avg_close_daily_df_2021, 
    sector_avg_close_daily_df_2022, 
    sector_avg_close_daily_df_2023, 
    sector_avg_close_daily_df_2024
])

### Testing the Hypothesis

#### Augmented Dickey-Fuller Test

In [88]:
adfuller(merged_daily_df_all['AvgClose'], autolag='AIC')

(-8.54259410207813,
 9.69242370717306e-14,
 53,
 36194,
 {'1%': -3.4305306864076686,
  '5%': -2.8616198590100925,
  '10%': -2.5668125064267433},
 430412.9772813759)

In [89]:
adfuller(merged_daily_df_all['EventCount'], autolag='AIC')

(-10.438116994091093,
 1.5471992886095674e-18,
 50,
 36197,
 {'1%': -3.4305306714313537,
  '5%': -2.861619852391126,
  '10%': -2.5668125029036415},
 717732.5166374843)

#### Granger Causality Test

The Granger causality test outputs results for different lags (1 to maxlag), showing whether the past values of EventCount help predict AvgClose. $\\$
If the p-value at a specific lag is significant (e.g., p < 0.05), it suggests that the effect takes that many days to show.

In [90]:
data_lag = merged_daily_df_all[['EventCount', 'AvgClose']].dropna()
granger_test_result = grangercausalitytests(data_lag, maxlag=30, verbose=True)

p_values = {lag: test[0]['ssr_ftest'][1] for lag, test in granger_test_result.items()}
p_values_df = pd.DataFrame(p_values.items(), columns=['Lag', 'p-value'])
p_values_df
# top_5_results = p_values_df.sort_values(by='p-value').head(5)
# print(top_5_results)

C:\Users\lklin\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning:

verbose is deprecated since functions should not print results




Granger Causality
number of lags (no zero) 1
ssr based F test:         F=7.6729  , p=0.0056  , df_denom=36244, df_num=1
ssr based chi2 test:   chi2=7.6736  , p=0.0056  , df=1
likelihood ratio test: chi2=7.6728  , p=0.0056  , df=1
parameter F test:         F=7.6729  , p=0.0056  , df_denom=36244, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.5714  , p=0.0103  , df_denom=36241, df_num=2
ssr based chi2 test:   chi2=9.1441  , p=0.0103  , df=2
likelihood ratio test: chi2=9.1429  , p=0.0103  , df=2
parameter F test:         F=4.5714  , p=0.0103  , df_denom=36241, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=3.7035  , p=0.0112  , df_denom=36238, df_num=3
ssr based chi2 test:   chi2=11.1126 , p=0.0111  , df=3
likelihood ratio test: chi2=11.1109 , p=0.0111  , df=3
parameter F test:         F=3.7035  , p=0.0112  , df_denom=36238, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=3.7495  

,Lag,p-value
0,1,0.005608
1,2,0.010349
2,3,0.011151
3,4,0.004711
4,5,0.008012
5,6,0.010353
6,7,0.017378
7,8,0.009171
8,9,0.006189
9,10,0.008896


In [94]:
# Collect p-values from the Granger test
lag_p_values = {lag: round(res[0]['ssr_ftest'][1], 4) for lag, res in granger_test_result.items()}

# Identify the lag with the lowest p-value (most significant)
best_lag = min(lag_p_values, key=lag_p_values.get)
print(f"The most significant lag is: {best_lag} days with p-value: {lag_p_values[best_lag]}")

The most significant lag is: 22 days with p-value: 0.0


In [95]:
significant_lags = [lag for lag, p in lag_p_values.items() if p < 0.05]

if significant_lags:
    avg_delay = sum(significant_lags) / len(significant_lags)
    print(f"Average delay: {avg_delay:.2f} days")
else:
    print("No significant lags found.")

Average delay: 13.00 days


### Summary

- Monthly Analysis
    - The Granger causality test revealed no significant relationship between event counts and average close prices.
- Daily Analysis
    - Provide the better insights
    - Most daily lags were significant
    - The average delay between an event and its impact on the stock market was approximately 13 days
